# Semantic Objects Tutorial

The purpose of this library is to demonstrate a simplified interface to ontologies in the built environment. It can be used to define objects that can feed all the basic processes we do using these ontologies - model creation, validation, and query. It can align model creation and usage, without the effort of creating custom RDF, SHACL, SPARQL. It also can be used to enable alignment of multiple ontologies to an application, by creating multiple compatible semantic objects for different ontologies. 

This library hopes to build on other projects. Currently, it produces templates that can be used by Semantic_Model_Builder to create models, create SHACL, validate and query data. It also intends to work with GraFrame and fuzzy matching of graphs to query data using a more modern workflow. 

## Ultimately, the functionality of this library will be three fold:

 - 1. A way to generate templates from semantic objects
 - 2. A way to generate SHACL/ontology statements from semantic objects for validation/inference
 - 3. A way to generate initialize semantic_objects from data graphs 

Currently, it is focused on generating template libraries, that can be used by other packages to demonstrate these use cases


In [17]:
# Select your ontology of choice, and some utilities to help export templates
from semantic_objects.s223 import *
from semantic_objects.core import export_templates
from pprint import pprint

In [9]:
# The semantic objects define entities, relations, or properties. 
# An entity may be something like a window
Window

semantic_objects.s223.entities.Window

In [10]:
# This window may have some properties (such as its Area, Tilt, or Azumith)
# These properties are defined as fields.
Window.__dataclass_fields__

{'area': Field(name='area',type=<class 'semantic_objects.s223.values.Area'>,default=<dataclasses._MISSING_TYPE object at 0x100c83790>,default_factory=<dataclasses._MISSING_TYPE object at 0x100c83790>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD),
 'azimuth': Field(name='azimuth',type=<class 'semantic_objects.s223.values.Azimuth'>,default=<dataclasses._MISSING_TYPE object at 0x100c83790>,default_factory=<dataclasses._MISSING_TYPE object at 0x100c83790>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD),
 'tilt': Field(name='tilt',type=<class 'semantic_objects.s223.values.Tilt'>,default=<dataclasses._MISSING_TYPE object at 0x100c83790>,default_factory=<dataclasses._MISSING_TYPE object at 0x100c83790>,ini

In [11]:
# Each of these fields is a property. These properties would be defined with values, units, quantitykinds, or other features
print(Area)
print(Area.__dataclass_fields__)

<class 'semantic_objects.s223.values.Area'>
{'qk': Field(name='qk',type=<class 'semantic_objects.qudt.quantitykinds.QuantityKind'>,default=<class 'semantic_objects.qudt.quantitykinds.Area'>,default_factory=<dataclasses._MISSING_TYPE object at 0x100c83790>,init=False,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD), 'value': Field(name='value',type=<class 'semantic_objects.core.Num'>,default=<dataclasses._MISSING_TYPE object at 0x100c83790>,default_factory=<dataclasses._MISSING_TYPE object at 0x100c83790>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD), 'unit': Field(name='unit',type=<class 'semantic_objects.units.Unit'>,default=<dataclasses._MISSING_TYPE object at 0x100c83790>,default_factory=<dataclasses._MISSING_TYPE object at 0x100c83790>,init=True,repr=True,hash=None,compare=True,metadata=mapping

In [14]:
# The properties can be exported as templates for use by buildingmotif and Semantic_Model_Builder
# This works by retrieving all the related classes to a class list, creating templates, 
# and exporting them to a directory (i.e. templates) following the convention of semantic_model_builder)
export_templates(Window, 'templates')

In [22]:
# We may also look at the related classes and an example of the body of the template 
related_classes = get_related_classes(Window)
print('related classes:')
pprint(related_classes)
print('\n example template:')
print(related_classes[-1][0].to_yaml())

related classes:
([<class 'semantic_objects.s223.relations.hasProperty'>,
  <class 'semantic_objects.s223.relations.hasUnit'>,
  <class 'semantic_objects.s223.relations.hasQuantityKind'>,
  <class 'semantic_objects.s223.relations.hasValue'>],
 [<class 'semantic_objects.s223.entities.Window'>],
 [<class 'semantic_objects.s223.values.Azimuth'>,
  <class 'semantic_objects.s223.values.Tilt'>,
  <class 'semantic_objects.s223.values.Area'>])

 example template:
Azimuth:
  body: |+
    @prefix P: <urn:___param___#> .
    @prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
    @prefix qudt: <http://qudt.org/schema/qudt/> .
    @prefix s223: <http://data.ashrae.org/standard223#> .

    P:name a s223:QuantifiableObservableProperty ;
        s223:hasUnit P:unit ;
        s223:hasValue P:value ;
        qudt:hasQuantityKind quantitykind:Azimuth .

  dependencies: []



In [ ]:
# These templates can be used with semantic_model_builder or buildingmotif to demonstrate functionality for model creation, query, validation.
# Creation of the SHACL (aka. declaration of the ontology for the semantic objects) is currently unfinished 

# we may see all the constraints for a particular class, including those coming from its parents
print(Space.generate_rdf_class_definition(include_hierarchy=True))

# or we may see just the constraints for a particular class, assuming we are using rdfs for inheritance within our ontology
print(Space.generate_rdf_class_definition(include_hierarchy=False))

@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

s223:PhysicalSpace a s223:Class,
        sh:NodeShape ;
    rdfs:label "Physical Space" ;
    rdfs:comment "A `PhysicalSpace` is an architectural concept representing a room, a part of a room, a collection of rooms, or any other physical region in a building. PhysicalSpaces may be grouped to define larger `PhysicalSpace`s using the relation `contains` (see {s223:contains})." ;
    rdfs:subClassOf s223:Class ;
    sh:property [ a sh:PropertyShape ;
            rdfs:comment "If the relation `encloses` is present it must associate the `Space` with a `DomainSpace`." ;
            sh:class s223:DomainSpace ;
            sh:message "s223: If the relation `encloses` is present it must associate the `Space` with a `DomainSpace`." ;
            sh:path s223:encloses ],
        [ a sh:Property